In [1]:
versioninfo()

Julia Version 1.11.1
Commit 8f5b7ca12a (2024-10-16 10:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 8 × Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, skylake)
Threads: 1 default, 0 interactive, 1 GC (on 8 virtual cores)
Environment:
  JULIA_DEPOT_PATH = E:\Users\tate\.julia
  JULIA_PYTHONCALL_EXE = python


In [2]:
using Pkg
Pkg.activate(".")

  Activating project at `E:\Users\tate\Repo\article-logm-preconditioning`


In [3]:
using Dates
using LinearAlgebra
using Printf
using SparseArrays

using MatrixDepot # test matrices
using ArnoldiMethod # to compute eigenvalues
using LinearMaps # to compute eigenvalues
using BenchmarkTools # to measure time

[ Info: verify download of index files...
[ Info: reading database


EOFError()


┌ Warning: recreating database file
└ @ MatrixDepot E:\Users\tate\.julia\packages\MatrixDepot\4S7Oa\src\download.jl:59
[ Info: reading index files
[ Info: adding metadata...
[ Info: adding svd data...
[ Info: writing database
┌ Warning: exception during initialization: 'KeyError(MatrixDepot)'
└ @ MatrixDepot E:\Users\tate\.julia\packages\MatrixDepot\4S7Oa\src\MatrixDepot.jl:125


In [4]:
function print_log(msg="")
    println("$(now())\t$(msg)")
    flush(stdout)
end

print_log (generic function with 2 methods)

In [5]:
include("src/SPDLogmPrecQuad.jl")
# using Revise
using ..SPDLogmPrecQuad

In [6]:
matnames = [
    "HB/nos4", # for check
    "MathWorks/Kuu",
    "Norris/fv3",
    "Lourakis/bundle1",
    "Boeing/crystm02",
    "ACUSIM/Pres_Poisson",
    "UTEP/Dubcova1",
    "Oberwolfach/gyro_m",
    "Pothen/bodyy5",
    "Pothen/bodyy6"
]

algorithms = [:GL, :DE, :PGL, :PDE];

In [7]:
function compute_extreme_eigenvalues(A)
    decomp, history = partialschur(A, nev=5, which=:LM, tol=1e-5, maxdim=50)
    λ_max = partialeigen(decomp)[1][5] |> Real
    
    F = factorize(A)
    function f!(y, F, x)
        y .= F \ x
        return y
    end
    lm = LinearMap{eltype(A)}((y, x) -> f!(y, F, x), size(A,1), issymmetric=true, isposdef=true)
    decomp, history = partialschur(lm, nev=5, which=:LM, tol=1e-5, maxdim=50)
    μ = partialeigen(decomp)[1]
    λ_min = 1 / μ[5]

    return λ_max, λ_min
end

compute_extreme_eigenvalues (generic function with 1 method)

In [8]:
print_log("start!")

m = length(matnames)
a = length(algorithms)
TimeData = zeros(m, a)
InvData = zeros(m, a)
ϵ = 1e-12
for (i, matname) in enumerate(matnames)
    print_log("======= $(matname) ========")
    A = matrixdepot(matname)
    λ_max, λ_min = compute_extreme_eigenvalues(A)
    κ = λ_max / λ_min
    A = A / sqrt(λ_max*λ_min)
    λ_max = sqrt(κ)
    λ_min = 1 / sqrt(κ)
    n = size(A, 1)
    b = ones(n) |> normalize

    print_log(@sprintf("n = %6d, κ = %.3e", n, κ))

    print_log("--- get #inv data ---")
    for (j, algorithm) in enumerate(algorithms)
        print_log("matname: $(matname), algorithm: $(algorithm)")
        if algorithm == :GL
            m = logm_gl(A, λ_max, λ_min, B=b, ϵ=ϵ, m=nothing).m
        elseif algorithm == :DE
            m = logm_de(A, λ_max, λ_min, B=b, ϵ=ϵ, m=nothing).m
        elseif algorithm == :PGL
            m = logm_pgl(A, λ_max, λ_min, B=b, ϵ=ϵ, m=nothing).m
        elseif algorithm == :PDE
            m = logm_pde(A, λ_max, λ_min, B=b, ϵ=ϵ, m=nothing).m
        end
        InvData[i,j] = m
    end

    print_log("--- get time data ---")
    for (j, algorithm) in enumerate(algorithms)
        print_log("matname: $(matname), algorithm: $(algorithm)")
        if algorithm == :GL
            b_result = @benchmark logm_gl($A, $λ_max, $λ_min, B=$b, ϵ=$ϵ, m=nothing)
        elseif algorithm == :DE
            b_result = @benchmark logm_de($A, $λ_max, $λ_min, B=$b, ϵ=$ϵ, m=nothing)
        elseif algorithm == :PGL
            b_result = @benchmark logm_pgl($A, $λ_max, $λ_min, B=$b, ϵ=$ϵ, m=nothing)
        elseif algorithm == :PDE
            b_result = @benchmark logm_pde($A, $λ_max, $λ_min, B=$b, ϵ=$ϵ, m=nothing)
        end
        TimeData[i,j] = mean(b_result).time / 1e9
    end
end

print_log("finished!")

2024-10-20T23:09:21.683	start!
2024-10-20T23:09:22.101	======= HB/nos4 ========
2024-10-20T23:09:33.572	n =    100, κ = 1.578e+03
2024-10-20T23:09:33.573	--- get #inv data ---
2024-10-20T23:09:33.589	matname: HB/nos4, algorithm: GL
2024-10-20T23:09:35.349	matname: HB/nos4, algorithm: DE
2024-10-20T23:09:35.683	matname: HB/nos4, algorithm: PGL
2024-10-20T23:09:35.793	matname: HB/nos4, algorithm: PDE
2024-10-20T23:09:35.962	--- get time data ---
2024-10-20T23:09:35.963	matname: HB/nos4, algorithm: GL
2024-10-20T23:09:47.442	matname: HB/nos4, algorithm: DE
2024-10-20T23:09:58.497	matname: HB/nos4, algorithm: PGL
2024-10-20T23:10:09.459	matname: HB/nos4, algorithm: PDE
2024-10-20T23:10:20.408	======= MathWorks/Kuu ========
2024-10-20T23:10:20.626	n =   7102, κ = 3.383e+04
2024-10-20T23:10:20.626	--- get #inv data ---
2024-10-20T23:10:20.627	matname: MathWorks/Kuu, algorithm: GL
2024-10-20T23:10:26.990	matname: MathWorks/Kuu, algorithm: DE
2024-10-20T23:10:30.312	matname: MathWorks/Kuu, alg

In [9]:
InvData

10×4 Matrix{Float64}:
  46.0  50.0  36.0  68.0
 100.0  64.0  54.0  84.0
  49.0  53.0  38.0  72.0
  41.0  48.0  34.0  70.0
  29.0  45.0  28.0  68.0
 179.0  76.0  74.0  94.0
 119.0  64.0  60.0  90.0
 244.0  81.0  86.0  96.0
  69.0  59.0  44.0  74.0
 122.0  68.0  60.0  90.0

In [10]:
TimeData

10×4 Matrix{Float64}:
  0.00324341   0.00344275   0.00250769   0.00467736
  6.17444      3.33882      3.16518      4.74693
  0.570365     0.653544     0.434923     0.858303
  1.89244      2.20696      1.54944      3.16249
  1.80016      2.74815      1.63308      4.11791
 37.303       13.8697      15.1301      17.7392
 10.0623       4.67034      4.7066       7.02706
 24.3615       6.97295      7.56155      8.50846
  1.9876       1.85471      1.29147      2.2437
  3.90966      2.37918      1.9956       2.99982

In [13]:
InvData = [
 46.0  50.0  36.0  68.0
 100.0  64.0  54.0  84.0
  49.0  53.0  38.0  72.0
  41.0  48.0  34.0  70.0
  29.0  45.0  28.0  68.0
 179.0  76.0  74.0  94.0
 119.0  64.0  60.0  90.0
 244.0  81.0  86.0  96.0
  69.0  59.0  44.0  74.0
 122.0  68.0  60.0  90.0
]
TimeData = [
  0.00324341   0.00344275   0.00250769   0.00467736
  6.17444      3.33882      3.16518      4.74693
  0.570365     0.653544     0.434923     0.858303
  1.89244      2.20696      1.54944      3.16249
  1.80016      2.74815      1.63308      4.11791
 37.303       13.8697      15.1301      17.7392
 10.0623       4.67034      4.7066       7.02706
 24.3615       6.97295      7.56155      8.50846
  1.9876       1.85471      1.29147      2.2437
  3.90966      2.37918      1.9956       2.99982
]

10×4 Matrix{Float64}:
  0.00324341   0.00344275   0.00250769   0.00467736
  6.17444      3.33882      3.16518      4.74693
  0.570365     0.653544     0.434923     0.858303
  1.89244      2.20696      1.54944      3.16249
  1.80016      2.74815      1.63308      4.11791
 37.303       13.8697      15.1301      17.7392
 10.0623       4.67034      4.7066       7.02706
 24.3615       6.97295      7.56155      8.50846
  1.9876       1.85471      1.29147      2.2437
  3.90966      2.37918      1.9956       2.99982

In [15]:
lines = []

line = "Matrix & " * join(algorithms, " & ") * "\\\\"
push!(lines, line)
for (i, matname) in enumerate(matnames)
    j_min = argmin(TimeData[i, :])
    elements = []
    for (j, algorithm) in enumerate(algorithms)
        m = @sprintf("%d", InvData[i, j])
        t = @sprintf("%.2f", TimeData[i,j])
        element = "$(t) ($(m))"
        if j == j_min
            element = "\\textbf{$(element)}"
        end
        push!(elements, element)
    end
    M = split(matname, "/")[end]
    line = "\\texttt{$(M)} & " * join(elements, " & ") * "\\\\"
    push!(lines, line)
end

print(join(lines, "\n"))

Matrix & GL & DE & PGL & PDE\\
\texttt{nos4} & 0.00 (46) & 0.00 (50) & \textbf{0.00 (36)} & 0.00 (68)\\
\texttt{Kuu} & 6.17 (100) & 3.34 (64) & \textbf{3.17 (54)} & 4.75 (84)\\
\texttt{fv3} & 0.57 (49) & 0.65 (53) & \textbf{0.43 (38)} & 0.86 (72)\\
\texttt{bundle1} & 1.89 (41) & 2.21 (48) & \textbf{1.55 (34)} & 3.16 (70)\\
\texttt{crystm02} & 1.80 (29) & 2.75 (45) & \textbf{1.63 (28)} & 4.12 (68)\\
\texttt{Pres_Poisson} & 37.30 (179) & \textbf{13.87 (76)} & 15.13 (74) & 17.74 (94)\\
\texttt{Dubcova1} & 10.06 (119) & \textbf{4.67 (64)} & 4.71 (60) & 7.03 (90)\\
\texttt{gyro_m} & 24.36 (244) & \textbf{6.97 (81)} & 7.56 (86) & 8.51 (96)\\
\texttt{bodyy5} & 1.99 (69) & 1.85 (59) & \textbf{1.29 (44)} & 2.24 (74)\\
\texttt{bodyy6} & 3.91 (122) & 2.38 (68) & \textbf{2.00 (60)} & 3.00 (90)\\